## BiLSTM com função de perda ponderada

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "drive/MyDrive/Mestrado/defesa/data/70-15-15"
model_path = "drive/MyDrive/Mestrado/defesa/models"

Mounted at /content/drive


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers


from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.metrics import matthews_corrcoef, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.utils import shuffle
import pickle
import matplotlib.pyplot as plt
import warnings
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
treino=pd.read_csv(data_path+'/comunic_treino.csv', encoding='utf-8')
teste=pd.read_csv(data_path+'/comunic_teste.csv', encoding='utf-8')
validacao=pd.read_csv(data_path+'/comunic_valid.csv', encoding='utf-8')
#dataset = load_dataset("csv", data_files={"train":data_path+'/comunic_treino.csv', "test": data_path+'/comunic_teste.csv', "validation":data_path+'/comunic_valid.csv'})

# Definição do modelo de classificacao BiLSTM

### Bag of words

In [ ]:
tok = Tokenizer(num_words=2000, oov_token='UNK')
tok.fit_on_texts(treino['text'] )
# Extract binary BoW features
X_train = tok.texts_to_sequences(treino['text'])
Y_train=treino.label
X_test = tok.texts_to_sequences(teste['text'])
Y_test=teste.label
X_val = tok.texts_to_sequences(validacao['text'])
Y_val=validacao.label

vocab_size = len(tok.word_index) + 1
vocab_size

11447

In [ ]:
maxlen = max([len(t) for t in X_train])
maxlen

3348

In [ ]:
training_padded = pad_sequences(X_train,
                                maxlen=maxlen, 
                                truncating='post', 
                                padding='post'
                               )
test_padded = pad_sequences(X_test,
                            maxlen=maxlen, 
                            truncating='post', 
                            padding='post'
                               )
val_padded = pad_sequences(X_val,
                            maxlen=maxlen, 
                            truncating='post', 
                            padding='post'
                               )

### Modelo

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(treino.label), y= treino.label)
class_weights = dict(zip(np.unique(treino.label), class_weights))
class_weights

{0: 1.2253521126760563,
 1: 1.0740740740740742,
 2: 5.117647058823529,
 3: 2.4857142857142858,
 4: 0.16353383458646617,
 5: 2.1219512195121952,
 6: 4.35,
 7: 1.359375,
 8: 5.4375,
 9: 0.8529411764705882,
 10: 2.806451612903226,
 11: 4.35,
 12: 0.8613861386138614}

In [ ]:
def cria_bilstm():

    model_bilstm = models.Sequential()
    model_bilstm.add(layers.Embedding(vocab_size, 512, input_length=maxlen))
    model_bilstm.add(layers.Bidirectional(layers.LSTM(256, dropout=0.25, recurrent_dropout=0, return_sequences=True)))
    #model_bilstm.add(layers.Bidirectional(layers.LSTM(128, dropout=0.25, recurrent_dropout=0, return_sequences=True)))
    #model_bilstm.add(layers.TimeDistributed(layers.Dense(128,activation='relu')))
    model_bilstm.add(layers.Flatten())
    #model_bilstm.add(layers.Dense(512, activation='relu'))
    model_bilstm.add(layers.Dense(13, activation='sigmoid'))
    model_bilstm.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model_bilstm.summary()
    return model_bilstm

In [ ]:
'''
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
'''

'\nfrom tensorflow.compat.v1 import ConfigProto\nfrom tensorflow.compat.v1 import InteractiveSession\nconfig = ConfigProto()\nconfig.gpu_options.allow_growth = True\nsession = InteractiveSession(config=config)\n'

In [ ]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping

early_stopping =  EarlyStopping(
    monitor='val_accuracy', 
    patience=4
)


In [ ]:
from scikeras.wrappers import KerasClassifier
estimator = KerasClassifier(model=cria_bilstm, epochs=50, batch_size=64)

In [ ]:
estimator.fit(training_padded, 
              Y_train, 
              batch_size=64, 
              epochs=50, 
              validation_data=(val_padded,Y_val), 
              class_weight=class_weights,
              callbacks=[early_stopping])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3348, 512)         5860864   
                                                                 
 bidirectional (Bidirectiona  (None, 3348, 512)        1574912   
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 1714176)           0         
                                                                 
 dense (Dense)               (None, 13)                22284301  
                                                                 
Total params: 29,720,077
Trainable params: 29,720,077
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
18/18 [==============================] - 36s 996ms/step - loss: 17.7494 - accuracy: 0.0663 - val_lo

KerasClassifier(
	model=<function cria_bilstm at 0x7f73f782adc0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=64
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=50
	class_weight=None
)

### Predição

In [ ]:
Y_pred = estimator.predict(test_padded)

4/4 [==============================] - 2s 287ms/step


### Métricas

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, hamming_loss
precision = precision_score(Y_test, Y_pred,average='weighted')
recall = recall_score(Y_test, Y_pred,average='weighted')
f1_micro_average = f1_score(y_true=Y_test, y_pred=Y_pred, average='weighted')
#roc_auc = roc_auc_score(Y_test, Y_pred, average = 'micro')
accuracy = accuracy_score(Y_test, Y_pred)
hloss = hamming_loss(Y_test, Y_pred)
print("Acurácia:",accuracy)
print("Precisão:",precision)
print("Recall:",recall)
print("F1:",f1_micro_average)
#print("Roc_auc:",roc_auc)
print("Hamming Loss:",hloss)

Acurácia: 0.757201646090535
Precisão: 0.7880093564257038
Recall: 0.757201646090535
F1: 0.7559431313397637
Hamming Loss: 0.24279835390946503
